In [1]:
#載入所需函示庫

from matplotlib import pyplot as plt
%matplotlib inline
# 設定圖形大小; DPI越大圖越大
plt.rcParams["figure.dpi"] = 120

import pandas as pd
import numpy as np
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.stats.api as sms
import statsmodels.formula.api as smf
import statsmodels.stats.multicomp as smm
import statsmodels

## Exercise 19.16  
### hypothesis (10%)  
H0: The two population locations are the same  
H1: The location of population 1 is different from the location of population 2  
population 1: day shift  
population 2: night shift  

In [5]:
#讀取資料集
df_xr1916 = pd.read_excel('Xr19-16.xlsx')
df_xr1916

,Day,Night
0,2,2.0
1,4,2.0
2,2,2.0
3,4,4.0
4,1,3.0
...,...,...
177,3,NaN
178,4,NaN
179,4,NaN
180,3,NaN


### p-value (or statistic value) (10%)

In [18]:
# Mann-Whitney U Test

stats.mannwhitneyu(df_xr1916['Day'],df_xr1916['Night'].dropna(),alternative = 'two-sided')

MannwhitneyuResult(statistic=15572.5, pvalue=0.392990186062222)

In [4]:
# Z-Test 

# 將所有數值放在一起排序
tmp1=df_xr1916.values.reshape(-1)
tmp1.sort()
tmp1

#做出Rank
df_xr1916_2=pd.DataFrame({'value':tmp1})
df_xr1916_2['rank'] = df_xr1916_2.index + 1
df_xr1916_2.head(10)

# 將同數值的Rank平均
value2rank=df_xr1916_2.groupby('value').mean().reset_index()
value2rank

# Compute Sum of Ranks
samp1=pd.DataFrame({'value': df_xr1916['Day'].values})
samp1=pd.merge(samp1,value2rank)
T=samp1['rank'].sum()

#compute mean and standard deviation
n1=df_xr1916['Day'].dropna().shape[0]
n2=df_xr1916['Night'].dropna().shape[0]
ET=n1*(n1+n2+1)/2 
SigmaT=(n1*n2*(n1+n2+1)/12) ** 0.5
z=(T-ET)/SigmaT

#compute p-value
#note that we are doing a two-tailed test
pvalue = 1 - stats.norm.cdf(z)
if pvalue > 0.5:
    pvalue = stats.norm.cdf(z)
pvalue = pvalue * 2

print("T (Sum of Ranks) = ", T)
print("E(T) = ", ET)
print("Sigma_T = ", SigmaT)
print("z value = ", z)
print("p-value = ", pvalue)

T (Sum of Ranks) =  32225.5
E(T) =  31486.0
Sigma_T =  924.861971683703
z value =  0.7995787724450891
p-value =  0.4239548904136825


### interpretation (10%)  
P-value=0.4240 (>0.05)  
Cannot reject H0  
There is not enough evidence to infer that the night and day shifts rate the service differently.  
## Exercise 19.45  
### hypothesis (10%)  
H0: The two population locations are the same  
H1: The location of population 1 is to the right of the location of population 2  
population 1: European car  
population 2: North American car  

In [6]:
#讀取資料集
df_xr1945 = pd.read_excel('Xr19-45.xlsx')
df_xr1945

,European,American
0,28,53
1,24,6
2,95,53
3,28,24
4,24,6
5,95,28
6,24,28
7,53,24
8,53,24
9,24,24


In [28]:
df_xr1945['diff']=df_xr1945['European']-df_xr1945['American']
ind1=~(df_xr1945['diff'] == 0)
diff1=df_xr1945['diff'][ind1].values
diff1

array([-25,  18,  42,   4,  18,  67,  -4,  29,  29,  18, -25,  18, -25,
        18,  25,  42,  -4,  42,  22,  29, -25,  71,  67])

In [9]:
# The Sign Test
x=np.sum(diff1>0)
n=np.size(diff1)
p=0.5
z=(x-n*p)/(n*p*(1-p))**0.5

#Right tail test
pvalue = 1 - stats.norm.cdf(z)

print("z value = ", z)
print("p-value = ", pvalue)

z value =  2.2936585546278225
p-value =  0.010905059571861164


In [40]:
df_test = df_xr1945[df_xr1945['European']!= df_xr1945['American']].reset_index()

In [32]:
stats.wilcoxon(df_test['European'], df_test['American'], alternative='greater')

WilcoxonResult(statistic=224.0, pvalue=0.0037260055541992188)

In [39]:
# Wilcoxon Signed-Rank Test
stats.wilcoxon(df_xr1945['European'], df_xr1945['American'], alternative='greater')

WilcoxonResult(statistic=224.0, pvalue=0.004348529900383603)

In [29]:
# z-test
diff1_abs = np.abs(diff1)
diff1_abs.sort()
diff1_abs = pd.DataFrame({'diff_abs': diff1_abs})
diff1_abs['rank'] = diff1_abs.index + 1
diffabs2rank = diff1_abs.groupby('diff_abs').mean().reset_index()

sum1 = pd.DataFrame({'diff': diff1, 'diff_abs': np.abs(diff1)})
sum1 = pd.merge(sum1, diffabs2rank)
ind2 = sum1['diff'].values > 0
T = sum1['rank'][ind2].sum()

n = sum1.shape[0]
ET = n*(n + 1) / 4 
SigmaT = (n*(n + 1) * (2 * n + 1) / 24) ** 0.5

z = (T - ET) / SigmaT
# Right tail test
pvalue = 1 - stats.norm.cdf(z)

print(f"T (Sum of Ranks) = {T:.4f}")
print(f"E(T) = {ET:.4f}")
print(f"Sigma_T = {SigmaT:.4f}")
print(f"z value = {z:.4f}")
print(f"p-value = {pvalue:.4f}")

T (Sum of Ranks) = 224.0000
E(T) = 138.0000
Sigma_T = 32.8786
z value = 2.6157
p-value = 0.0045


### a. (10%)  
P-value=0.0109 (<0.05)  
Reject H0  
There is enough evidence to infer that the European car is perceived to be more comfortable.  
### b. (10%)  
The results are identical. In sign test, we only care about the sign of the difference. Therefore, all codes that preserve the order produce the same results.
## Exercise 19.56  
### hypothesis (10%)  
H0: The two population locations are the same  
H1: The location of population 1 is to the right of the location of population 2  
population 1: Grades of students from High School 1  
population 2: Grades of students from High School 2

In [19]:
#讀取資料集
df_xr1956 = pd.read_excel('Xr19-56.xlsx')
df_xr1956

,High School 1,High School 2
0,2,3
1,4,3
2,3,2
3,3,3
4,3,3
...,...,...
95,4,4
96,2,1
97,3,3
98,2,3


In [20]:
df_xr1956['diff']=df_xr1956['High School 1']-df_xr1956['High School 2']
ind1=~(df_xr1956['diff'] == 0)
diff1=df_xr1956['diff'][ind1].values
diff1

array([-1,  1,  1, -1, -1, -1,  1,  1,  1, -1,  1, -2,  1,  1, -1, -1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1,  1,  1, -1,  1, -1, -1,
        1, -1,  1, -1, -1,  1, -1, -1, -1,  1,  1, -1,  1, -1,  1,  1,  1,
        1, -1])

### p-value (or statistic value) (10%)

In [21]:
# The Sign Test

x=np.sum(diff1>0)
n=np.size(diff1)
p=0.5
z=(x-n*p)/(n*p*(1-p))**0.5

#Right tail test
pvalue = 1 - stats.norm.cdf(z)

print("z value = ", z)
print("p-value = ", pvalue)

z value =  1.5109662034355793
p-value =  0.06539853090342929


In [22]:
# Wilcoxon Signed-Rank Test
stats.wilcoxon(df_xr1956['High School 1'], df_xr1956['High School 2'], alternative='greater')

WilcoxonResult(statistic=848.0, pvalue=0.09072460386071024)

In [23]:
# z-test
diff1_abs = np.abs(diff1)
diff1_abs.sort()
diff1_abs = pd.DataFrame({'diff_abs': diff1_abs})
diff1_abs['rank'] = diff1_abs.index + 1
diffabs2rank = diff1_abs.groupby('diff_abs').mean().reset_index()

sum1 = pd.DataFrame({'diff': diff1, 'diff_abs': np.abs(diff1)})
sum1 = pd.merge(sum1, diffabs2rank)
ind2 = sum1['diff'].values > 0
T = sum1['rank'][ind2].sum()

n = sum1.shape[0]
ET = n*(n + 1) / 4 
SigmaT = (n*(n + 1) * (2 * n + 1) / 24) ** 0.5

z = (T - ET) / SigmaT
# Right tail test
pvalue = 1 - stats.norm.cdf(z)

print(f"T (Sum of Ranks) = {T:.4f}")
print(f"E(T) = {ET:.4f}")
print(f"Sigma_T = {SigmaT:.4f}")
print(f"z value = {z:.4f}")
print(f"p-value = {pvalue:.4f}")

T (Sum of Ranks) = 848.0000
E(T) = 715.5000
Sigma_T = 112.9591
z value = 1.1730
p-value = 0.1204


### interpretation (10%)  
P-value=0.0654 (>0.05)  
Cannot reject H0  
There is not enough evidence to infer that preference should be given to students for high school 1.